In [2]:
!pip install selenium

     |████████████████████████████████| 904 kB 503 kB/s eta 0:00:01


In [3]:
import selenium
from selenium import webdriver
import time
import pandas as pd
import json
import os

## Extract Links

In [6]:
def extract_links():
    # To extract links from given list of category urls and store in csv
    driver = webdriver.Chrome('/home/atul/scraping_driver/chromedriver')
    driver.maximize_window()
    time.sleep(5)
    comp_categ_list = []
    comp_list_link = []
    hyperlinks = ['https://www.alibaba.com/Medical-Devices_p100002906?spm=a27aq.13923566.9010494120.12.1d7231a8rQzV1u','https://www.alibaba.com/Medical-Consumables_p127866004?spm=a27aq.13923566.9010494120.13.1d7231a8rQzV1u','https://www.alibaba.com/Health-Care-Products_p100002908?spm=a27aq.13923566.9010494120.16.1d7231a8rQzV1u']
    i=0
    for link in hyperlinks:
        print(link)
        if i == 0:
            categ_str = 'Medical Devices'
        elif i == 1:
            categ_str = 'Medical Consumables'
        elif i == 2:
            categ_str = 'Health Care Products'
        driver.get(link)
        time.sleep(10)
        SCROLL_PAUSE_TIME = 0.7

        # Get scroll height
        last_height = driver.execute_script("return document.body.scrollHeight")+100

        while True:
            # Scroll down to bottom
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            # Wait to load page
            time.sleep(SCROLL_PAUSE_TIME)

            # Calculate new scroll height and compare with last scroll height
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                break
            last_height = new_height
        time.sleep(5)
        elements = driver.find_elements_by_class_name('product-detail')
        print(len(elements))
        links = set()
        for element in elements:
            product_link = element.get_attribute('href')
            if product_link not in links:
                links.add(product_link)
            else:
                continue
        list_links = list(links)
        print("Category",i,len(list_links))
        comp_list_link.extend(list_links)
        comp_categ_list.extend([categ_str]*len(list_links))
        i+=1
    driver.quit()
    df = pd.DataFrame(zip(comp_categ_list,comp_list_link),columns = ['category','product_urls'])
    print(df)
    df.to_csv('alibaba_product_urls1.csv',index = None)

In [7]:
extract_links()

https://www.alibaba.com/Medical-Devices_p100002906?spm=a27aq.13923566.9010494120.12.1d7231a8rQzV1u
180
Category 0 60
https://www.alibaba.com/Medical-Consumables_p127866004?spm=a27aq.13923566.9010494120.13.1d7231a8rQzV1u
420
Category 1 140
https://www.alibaba.com/Health-Care-Products_p100002908?spm=a27aq.13923566.9010494120.16.1d7231a8rQzV1u
378
Category 2 126
                 category                                       product_urls
0         Medical Devices  https://www.alibaba.com/product-detail/3-M-Ket...
1         Medical Devices  https://www.alibaba.com/product-detail/Lightsp...
2         Medical Devices  https://www.alibaba.com/product-detail/Best-Qu...
3         Medical Devices  https://www.alibaba.com/product-detail/5mm-and...
4         Medical Devices  https://www.alibaba.com/product-detail/HD11XE-...
..                    ...                                                ...
321  Health Care Products  https://www.alibaba.com/product-detail/Hot-sel...
322  Health Care Produ

In [11]:
df = pd.read_csv('alibaba_product_urls1.csv') #update path of product_urls.csv

In [12]:
df.head()

,category,product_urls
0,Medical Devices,https://www.alibaba.com/product-detail/3-M-Ket...
1,Medical Devices,https://www.alibaba.com/product-detail/Lightsp...
2,Medical Devices,https://www.alibaba.com/product-detail/Best-Qu...
3,Medical Devices,https://www.alibaba.com/product-detail/5mm-and...
4,Medical Devices,https://www.alibaba.com/product-detail/HD11XE-...


In [13]:
len(df)

326

In [14]:
def get_company_overview(driver):
    keys = [i.text for i in driver.find_elements_by_css_selector('.field-title')]
    values = [i.get_attribute('title') if i.get_attribute('title')!='' else i.text for i in driver.find_elements_by_css_selector('.content-value') ]
    return dict(zip(keys,values))

In [15]:
def get_production_capacity(driver):
    prod_cap_dic = {}
    prod_cap_dic['factory_info'] = {}
    keys = [i.text for i in driver.find_elements_by_css_selector('.icbu-shop-table-col-item .title div')]
    values = [i.text for i in driver.find_elements_by_css_selector('.icbu-shop-table-col-item .content span')]
    prod_cap_dic['factory_info'] = dict(zip(keys,values))
    return prod_cap_dic

In [16]:
def get_rd_capacity(driver):
    rd_capacity = {}
    rd_capacity['certification'] = []
    keys = ['Picture','Certification Name','Issued By','Business Scope','Available Date','Verified']
    li = driver.find_elements_by_css_selector('.icbu-pc-cpRDCapacity .next-table-cell .next-table-cell-wrapper')
    val = []
    for element in li:
        if element.text=='':
            try:
                new_val = element.find_element_by_tag_name('img').get_attribute('src')
            except:
                new_val = element.text
        else:
            new_val = element.text
        val.append(new_val)
    i=0
    while i< len(val):
        d = dict(zip(keys,val[i:i+6]))
        i+=6
        rd_capacity['certification'].append(d)
    #rd_capacity['no_of_people'] = [i.text for i in driver.find_elements_by_css_selector('.icbu-pc-cpRDCapacity .infoList-mod-field~ .infoList-mod-field+ .infoList-mod-field .content div')][-1]
    return rd_capacity

In [17]:
def get_trade_capacity(driver):
    tr_capacity = {}
    tr_capacity['main_markets and products'] = []
    keys = ['Main Markets', 'Total Revenue(%)','Main Product(s)','Verified']
    li = [i.text for i in driver.find_elements_by_css_selector('.icbu-pc-cpTradeCapability .next-table-cell .next-table-cell-wrapper')]
    i=0
    while i< len(li):
        d = dict(zip(keys,li[i:i+4]))
        i+=4
        tr_capacity['main_markets and products'].append(d)
    return tr_capacity

In [18]:
def get_product_details(browser):
    product_details = {}
    try:
        title = browser.find_element_by_class_name("ma-title")
        product_details["title"] = title.text
        print("-------1-------")
    except Exception as e:
        print(e)

    try:
        icons = browser.find_element_by_class_name("util-clearfix")
        icon_list = icons.find_elements_by_css_selector("img")
        images_list = []
        for icon in icon_list:
#                 icon.click()
            images_list.append(str(icon.get_attribute("src")).replace("_50x50.jpg",""))
        product_details["product_images"] = images_list
        print("-------2-------")
    except Exception as e:
        print(e)

#     details = browser.find_element_by_class_name("module-productPackagingAndQuickDetail")
#     print(details.text)

    overview = browser.find_element_by_class_name("do-overview")
    entries = overview.find_elements_by_class_name("do-entry")
    overview_dict = {}
    for entry in entries:
        title = entry.find_element_by_class_name("do-entry-title")
        overview_list = entry.find_element_by_class_name("do-entry-list")
        entry_items = overview_list.find_elements_by_tag_name("dl")
        item_dict = {}
        for item in entry_items:
            try:
                item_list = str(item.text).splitlines()
                item_dict[item_list[0]] = item_list[1]
            except Exception as e:
                print("List",e)
                continue
            print("-------3-------")
        overview_dict[title.text] = item_dict
    product_details["overview"] = overview_dict
 ###############################################################################################################   
    
    product_desc = {}
    try:
        child_eles = browser.find_elements_by_xpath("//*[@id='J-rich-text-description']/div")
        for child_div in child_eles:
            data_section = child_div.get_attribute("data-section-title")
            print(data_section)
            if data_section =="Product Description":
                print(data_section)
                try:
                    tables = child_div.find_elements_by_tag_name('table')[1]
                    table1_dict = {}
                    tr_list = tables.find_elements_by_tag_name('tr')
                    for tr in tr_list:
                        td = tr.find_elements_by_tag_name('td')
                        table1_dict[td[0].text] = td[1].text
                        print("-------4-------",table1_dict)
                        product_desc = table1_dict
                except Exception as e:
                    # Not a table, handling for image
                    print(e)
                # For images:
                
            elif data_section=="User Manual":
                manual_img = []
                try:
                    user_manual = child_div
                    user_manual_img = user_manual.find_element_by_tag_name('img')
                    manual_img.append(user_manual_img.get_attribute('data-src'))
                    print("")
                    print("-------7-------")
                except Exception as e:
                    print("User Manual:",e)
            elif data_section=="Company Overview":
                try:
                    company_overview = child_div
                    company_overview_imgs = company_overview.find_elements_by_tag_name('img')
                    for img in company_overview_imgs:
                        manual_img.append(img.get_attribute('data-src'))
                        print("-------8-------")
                    product_details["user_manual"] = manual_img
                except Exception as e:
                    print("Company Overview:",e)
            elif data_section=="Our Partners":                
                try:
                    partner = child_div
                    partner_imgs = partner.find_elements_by_tag_name('img')
                    for img in partner_imgs:
                        product_details["partner"] = img.get_attribute('data-src')
                        print("-------9-------")
                except:
                    print("Our Partners:",e)
            elif data_section=="Our Services":                
                try:
                    our_service = {}
                    services = child_div

                    service_table = services.find_elements_by_tag_name("table")[1]
                    tr_list = service_table.find_elements_by_tag_name('tr')
                    service_dict = {}
                    for tr in tr_list:
                        td = tr.find_elements_by_tag_name('td')
                        service_dict[td[0].text] = td[1].text
                    our_service["description"] = service_dict

                    service_img = services.find_element_by_tag_name('img')
                    our_service["images"] = service_img.get_attribute('data-src')

                    product_details["our_services"] = our_service
                except Exception as e:
                    print("Our services:",e)
            elif data_section=="Certifications":
                try:
                    certificate = child_div
                    certificate_img = certificate.find_element_by_tag_name('img')
                    product_details["certificate"] = certificate_img.get_attribute('data-src')
                except Exception as e:
                    print("certificate:",e)
            elif data_section=="Packing & Delivery":                
                try:
                    pack_delv_dict = {}
                    pack_deliv = child_div
                    pack_deliv_imgs = pack_deliv.find_elements_by_tag_name('img')
                    pack_delv_dict["packing"] = pack_deliv_imgs[0].get_attribute('data-src')
                #         print('---paking----',packing)

                    pack_delv_dict["delivery"] = pack_deliv_imgs[1].get_attribute('data-src')
                #         print('--delivery---',delivery)
                    delivery_table = pack_deliv.find_elements_by_tag_name("table")[1]
                    tr_list = delivery_table.find_elements_by_tag_name('tr')
                    delivery_dict = {}
                    for tr in tr_list:
                        td = tr.find_elements_by_tag_name('td')
                        delivery_dict[td[0].text] = td[1].text
                    pack_delv_dict["description"] = delivery_dict
                    product_details["packing_delivery"] = pack_delv_dict
                except Exception as e:
                    print("packaging:",e)
            elif data_section=="FAQ":
                try:
                    faqs = child_div
                #     print(faqs.text)
                    faq_list = faqs.find_elements_by_tag_name('span')
                    #if len(faq_list)==1:
                        
                    faq_dict = {}
                    for faq in faq_list:
                        if ":" in faq.text:
                            faq_qa = str(faq.text).split(":")
                            faq_dict[faq_qa[0]] = faq_qa[1]
                    product_details["FAQ"] = faq_dict
                except Exception as e:
                    print("faq:",e)
        try:
            product_details['image_content']={}
            dic = {}
            for p in child_eles:
                for ele in p.find_elements_by_tag_name('img'):
                    src = ele.get_attribute('data-src')
                    if src:
                        dic[p.get_attribute('data-section-title')]='https:'+src
                product_details['image_content']=dic
        except Exception as e:
            print("image content",e)
        return product_details
    except:
        print("Product desc not found !")  
        return product_details                  

In [19]:
def get_product_company_details(driver):
    dic = {}
    dic['product_details']  = {}
    dic['product_details'] = get_product_details(driver)
    # company
    try:
        driver.find_element_by_css_selector('.active+ .next-tabs-nav-appear-active .tab-name').click()
        dic['company_profile'] = {}
        dic['company_profile']['company_overview'] = dict()
        dic['company_profile']['production_capacity'] = dict()
        dic['company_profile']['r&d_capacity'] = dict()
        dic['company_profile']['trade_capacity'] = dict()
        
        dic['company_profile']['company_overview'] = get_company_overview(driver)
        dic['company_profile']['production_capacity'] = get_production_capacity(driver)
        try:
            driver.find_element_by_css_selector('.icbu-pc-cpRDCapacity .view-more').click()
            dic['company_profile']['r&d_capacity'] = get_rd_capacity(driver)
        except Exception as e:
        #J-ls-grid-desc > div.tab-body > div.tab-body-pane.ls-icon.ls-company.show > div.alisite > div:nth-child(5) > div > div > div > div.mod-content > div > div.mod-view-more > div
            print("R&D",e)
        try:
            driver.find_element_by_css_selector('.icbu-pc-cpTradeCapability .view-more').click()
            dic['company_profile']['trade_capacity'] = get_trade_capacity(driver)
        except Exception as e:
            print("Trade",e)
    except Exception as e:
        print("Company",e)
    return dic

In [25]:
def get_json_for_url(url):
    # To extract content from a product url
    driver = webdriver.Chrome('/home/atul/scraping_driver/chromedriver')
    driver.maximize_window()
    time.sleep(5)
    driver.get(url)
    time.sleep(5)
    product_name = driver.find_element_by_class_name('ma-title').get_attribute('title')
    SCROLL_PAUSE_TIME = 0.7

    # Get scroll height
    last_height = driver.execute_script("return document.body.scrollHeight")+100

    while True:
        # Scroll down to bottom
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    time.sleep(10)
    dic = get_product_company_details(driver)
    driver.quit()
    with open('crawler_output_files/result_'+product_name+'.json', 'w') as fp:
        json.dump(dic, fp)
    return product_name,dic

In [35]:
def store_multiple_json_files(df,n=len(df)):
    beg = time.time()
    print(n)
    full_json = {}
    product_names = []
    product_urls = []
    file_names = []
    count = 0 
    i = 0
    while count<n and i<len(df):
        
        try:
            print(df['product_urls'][i])
            product_name,dic = get_json_for_url(df['product_urls'][i])
            product_names.append(product_name)
            product_urls.append(df['product_urls'][i])
            file_names.append('result_'+product_name+'.json')
            full_json[i]={}
            count+=1
        except Exception as e:
            print("Page not found",e)
            #full_json[i]='Not Found'
            dic = 'Page not found'
        full_json[i]=dic
        i+=1
    output_df = pd.DataFrame({'product_name':product_names,'product_url':product_urls,'file_name':file_names})
    print("Time taken for ",n,"urls:",(time.time()-beg),"seconds")
    return output_df,full_json

In [36]:
t = time.time()
for i in range(10):
    print(i)
time.time()-t

0
1
2
3
4
5
6
7
8
9


0.002186298370361328

## Crawl Links

In [ ]:
begin = time.time()
output_df,full_json = store_multiple_json_files(df,10)

10
https://www.alibaba.com/product-detail/3-M-Ketac-Molar-Aplicap-A3_62009688882.html
-------1-------
-------2-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
R&D Message: no such element: Unable to locate element: {"method":"css selector","selector":".icbu-pc-cpRDCapacity .view-more"}
  (Session info: chrome=87.0.4280.66)

Trade Message: no such element: Unable to locate element: {"method":"css selector","selector":".icbu-pc-cpTradeCapability .view-more"}
  (Session info: chrome=87.0.4280.66)

Page not found [Errno 2] No such file or directory: 'crawler_output_files/result_3 M Ketac Molar Aplicap A3 (50ea).json'
https://www.alibaba.com/product-detail/Lightspeed-16-CT-meter-Scanner_62363304749.html
-------1-------
-------2-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
R&D Message: no such element: Unable to locate element: {"met

-------3-------
-------3-------
-------3-------
List list index out of range
-------3-------
None
R&D Message: no such element: Unable to locate element: {"method":"css selector","selector":".icbu-pc-cpRDCapacity .view-more"}
  (Session info: chrome=87.0.4280.66)

Trade Message: no such element: Unable to locate element: {"method":"css selector","selector":".icbu-pc-cpTradeCapability .view-more"}
  (Session info: chrome=87.0.4280.66)

Page not found [Errno 2] No such file or directory: 'crawler_output_files/result_Stainless Steel Adson Brown 12cm Tweezers High Quality Adson Brown Tweezers Dental Adson Brown Tweezers.json'
https://www.alibaba.com/product-detail/Medical-swab-supplies-genetic-testing-equipment_1600077495948.html
-------1-------
-------2-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3

-------1-------
-------2-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
List list index out of range
-------3-------
Product Description
Product Description
list index out of range
R&D Message: no such element: Unable to locate element: {"method":"css selector","selector":".icbu-pc-cpRDCapacity .view-more"}
  (Session info: chrome=87.0.4280.66)

Page not found [Errno 2] No such file or directory: 'crawler_output_files/result_Best Sterile Medical Products Coronary Balloon and Stents for Angioplasty.json'
https://www.alibaba.com/product-detail/Dental-Extracting-Forceps-Upper-Lower-Molars_1600137524205.html
-------1-------
-------2-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3----

-------1-------
-------2-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
None
R&D Message: no such element: Unable to locate element: {"method":"css selector","selector":".icbu-pc-cpRDCapacity .view-more"}
  (Session info: chrome=87.0.4280.66)

Trade Message: no such element: Unable to locate element: {"method":"css selector","selector":".icbu-pc-cpTradeCapability .view-more"}
  (Session info: chrome=87.0.4280.66)

Page not found [Errno 2] No such file or directory: 'crawler_output_files/result_Simple style modern dental handpiece CHIRANA TGL 656 EASY turbine handpiece.json'
https://www.alibaba.com/product-detail/Hot-sale-upright-large-2-8_1600135587279.html
-------1-------
-------2-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
-------3-------
------

In [29]:
full_json

{0: 'Page not found',
 1: 'Page not found',
 2: 'Page not found',
 3: 'Page not found',
 4: 'Page not found',
 5: 'Page not found',
 6: 'Page not found',
 7: 'Page not found',
 8: 'Page not found',
 9: 'Page not found',
 10: 'Page not found',
 11: 'Page not found',
 12: 'Page not found',
 13: 'Page not found',
 14: 'Page not found',
 15: 'Page not found',
 16: 'Page not found',
 17: 'Page not found',
 18: 'Page not found',
 19: 'Page not found',
 20: 'Page not found',
 21: 'Page not found',
 22: 'Page not found',
 23: 'Page not found',
 24: 'Page not found',
 25: 'Page not found',
 26: 'Page not found',
 27: 'Page not found',
 28: 'Page not found',
 29: 'Page not found',
 30: 'Page not found',
 31: 'Page not found',
 32: 'Page not found',
 33: 'Page not found',
 34: 'Page not found',
 35: 'Page not found',
 36: 'Page not found',
 37: 'Page not found',
 38: 'Page not found',
 39: 'Page not found',
 40: 'Page not found',
 41: 'Page not found',
 42: 'Page not found',
 43: 'Page not found'

In [39]:
ldf = pd.DataFrame(full_json).T

In [50]:
ldf.to_csv('file.csv',index = None)

In [48]:
ldf['product_details'][0]

{'title': 'CE ISO and FDA approved non contact infrared waterproof digital thermometer',
 'product_images': ['https://img.alicdn.com/imgextra/i3/6000000003165/O1CN01uIncQB1ZFc6E7eqbh_!!6000000003165-0-tbvideo.jpg',
  'https://sc02.alicdn.com/kf/HTB1mrG6abr1gK0jSZR0q6zP8XXaG.jpg',
  'https://sc01.alicdn.com/kf/HTB1Hti6akH0gK0jSZPiq6yvapXaE.jpg',
  'https://sc01.alicdn.com/kf/HTB1ybG6abr1gK0jSZR0q6zP8XXa5.jpg',
  'https://sc02.alicdn.com/kf/HTB1tUO5akL0gK0jSZFtq6xQCXXax.jpg',
  'https://sc02.alicdn.com/kf/HTB1z6a5ahz1gK0jSZSgq6yvwpXa2.jpg',
  'https://sc01.alicdn.com/kf/HTB1BZK5aoD1gK0jSZFGq6zd3FXaV.jpg'],
 'overview': {'Quick Details': {'Type:': 'Thermometer',
   'Brand Name:': 'OREINTMED, ORIENTMED',
   'Place of Origin:': 'Fujian, China',
   'Model Number:': 'ORT601',
   'Instrument classification:': 'Class II',
   'Warranty:': '1 YEAR',
   'After-sale Service:': 'Online Technical Support',
   'Memory:': '32*2 sets',
   'Auto power off:': '8 seconds',
   'Display:': 'LCD',
   'Color:'

In [51]:
ldf

,product_details,company_profile
0,{'title': 'CE ISO and FDA approved non contact...,{'company_overview': {'Business Type': 'Tradin...
1,{'title': 'Multi-purpose Mini UVC Light Mobile...,{'company_overview': {'Business Type': 'Manufa...
2,{'title': 'In Stock UVC LED Sterilization Bag ...,{'company_overview': {'Business Type': 'Manufa...
3,{'title': 'Highest configuration ICU ventilato...,{'company_overview': {'Business Type': 'Manufa...
4,{'title': 'STOCK Surgical Mask N95 Medical Por...,{'company_overview': {'Business Type': 'Manufa...
...,...,...
462,{'title': '2020 Latest Intelligent Electric Wi...,{'company_overview': {'Business Type': 'Manufa...
463,{'title': 'Jinghao Disability Aids Hearing Aid...,{'company_overview': {'Business Type': 'Manufa...
464,{'title': 'Hot car buttocks vibration heated m...,{'company_overview': {'Business Type': 'Tradin...
465,{'title': 'Hot Sell Internal Rechargeable Digi...,{'company_overview': {'Business Type': 'Manufa...


In [56]:
full_json[0]

{'product_details': {'title': 'CE ISO and FDA approved non contact infrared waterproof digital thermometer',
  'product_images': ['https://img.alicdn.com/imgextra/i3/6000000003165/O1CN01uIncQB1ZFc6E7eqbh_!!6000000003165-0-tbvideo.jpg',
   'https://sc02.alicdn.com/kf/HTB1mrG6abr1gK0jSZR0q6zP8XXaG.jpg',
   'https://sc01.alicdn.com/kf/HTB1Hti6akH0gK0jSZPiq6yvapXaE.jpg',
   'https://sc01.alicdn.com/kf/HTB1ybG6abr1gK0jSZR0q6zP8XXa5.jpg',
   'https://sc02.alicdn.com/kf/HTB1tUO5akL0gK0jSZFtq6xQCXXax.jpg',
   'https://sc02.alicdn.com/kf/HTB1z6a5ahz1gK0jSZSgq6yvwpXa2.jpg',
   'https://sc01.alicdn.com/kf/HTB1BZK5aoD1gK0jSZFGq6zd3FXaV.jpg'],
  'overview': {'Quick Details': {'Type:': 'Thermometer',
    'Brand Name:': 'OREINTMED, ORIENTMED',
    'Place of Origin:': 'Fujian, China',
    'Model Number:': 'ORT601',
    'Instrument classification:': 'Class II',
    'Warranty:': '1 YEAR',
    'After-sale Service:': 'Online Technical Support',
    'Memory:': '32*2 sets',
    'Auto power off:': '8 seconds

In [29]:
output_df.to_csv('final_df.csv',index = None)

In [112]:
list_size = len(full_json.keys())
title_lst = [None]*list_size
product_img_lst = [None]*list_size
overview_lst = [None]*list_size
our_services = [None]*list_size
packaging_delivery = [None]*list_size
faq = [None]*list_size
image_content = [None]*list_size
company_overview = [None]*list_size
production_capacity = [None]*list_size
r_d_capacity = [None]*list_size
trade_capacity = [None]*list_size
user_manual = [None]*list_size
for key in full_json.keys():
    if full_json[key]!='Page not found':
        title_lst[key] = str(full_json[key]['product_details']['title'])
        product_img_lst[key] = str(full_json[key]['product_details']['product_images'])
        overview_lst[key] = str(full_json[key]['product_details']['overview'])
        if full_json[key]['product_details'].get('user_manual'):
            user_manual[key] = str(full_json[key]['product_details']['user_manual'])
        if full_json[key]['product_details'].get('our_services'):
            our_services[key] = str(full_json[key]['product_details']['our_services'])

        if full_json[key]['product_details']['overview'].get('Packaging & Delivery'):
            packaging_delivery[key] = str(full_json[key]['product_details']['overview']['Packaging & Delivery'])

        if full_json[key]['product_details'].get('FAQ'):   
            faq[key] = str(full_json[key]['product_details']['FAQ'])


        image_content[key] = str(full_json[key]['product_details']['image_content'])

        company_overview[key] = str(full_json[key]['company_profile']['company_overview'])
        production_capacity[key] = str(full_json[key]['company_profile']['production_capacity'])
        r_d_capacity[key] = str(full_json[key]['company_profile']['r&d_capacity'])
        trade_capacity[key] = str(full_json[key]['company_profile']['trade_capacity'])

pd 0
pd 1
pd 2
pd 3
pd 4
pd 5
pd 7
s 8
pd 8
faq 8
pd 9
pd 10
pd 12
pd 13
pd 14
pd 15
pd 17
pd 18
pd 19
pd 20
pd 21
faq 21
pd 22
pd 23
pd 24
pd 25
pd 27
pd 28
pd 29
pd 30
pd 31
pd 32
pd 33
pd 34
pd 36
pd 37
pd 38
pd 39
pd 43
pd 44
pd 45
pd 46
pd 47
pd 48
pd 49
pd 50
pd 51
pd 52
pd 54
pd 55
pd 56
pd 57
pd 58
pd 59
pd 60
pd 61
pd 62
pd 63
pd 64
pd 65
pd 66
pd 67
pd 68
pd 69
pd 70
pd 71
pd 73
pd 74
pd 75
pd 76
faq 76
pd 77
faq 77
pd 80
pd 81
pd 82
pd 83
pd 84
pd 85
pd 86
pd 87
pd 88
pd 89
faq 89
pd 90
pd 92
pd 93
pd 94
pd 95
pd 96
pd 98
pd 99
pd 101
pd 102
pd 103
pd 105
pd 107
pd 108
pd 110
faq 110
pd 111
pd 112
pd 113
pd 114
pd 115
pd 116
pd 117
pd 118
pd 119
pd 121
pd 122
pd 123
pd 124
faq 124
pd 125
pd 127
pd 128
faq 128
pd 129
pd 130
pd 131
pd 132
pd 133
pd 134
pd 135
pd 136
pd 137
pd 139
pd 140
pd 141
pd 142
faq 142
pd 143
pd 145
pd 146
pd 147
pd 148
pd 149
faq 149
pd 150
pd 152
pd 153
pd 154
pd 155
pd 157
pd 158
pd 159
pd 160
pd 161
pd 162
pd 163
faq 163
pd 164
pd 165
s 166
pd 166
fa

In [113]:
df_dic = {'product_title':title_lst,
 'product_images':product_img_lst,
 'overview':overview_lst,
 'user_manual':user_manual,
 'our_services':our_services,
 'packaging_delivery':packaging_delivery,
 'FAQ': faq,
 'image_content':image_content,
 'company_overview':company_overview,
 'production_capacity':production_capacity,
 'r&d_capacity':r_d_capacity,
 'trade_capacity':trade_capacity
}

In [125]:
final_df1 = pd.DataFrame(df_dic)

In [126]:
final_df1.isna().sum()

product_title           61
product_images          61
overview                61
user_manual            465
our_services           465
packaging_delivery      62
FAQ                    419
image_content           61
company_overview        61
production_capacity     61
r&d_capacity            61
trade_capacity          61
dtype: int64

In [127]:
final_df1['product_url'] = df['product_urls']

In [129]:
final_df1['category'] = df['category']

In [130]:
final_df1

,product_title,product_images,overview,user_manual,our_services,packaging_delivery,FAQ,image_content,company_overview,production_capacity,r&d_capacity,trade_capacity,product_url,category
0,CE ISO and FDA approved non contact infrared w...,['https://img.alicdn.com/imgextra/i3/600000000...,"{'Quick Details': {'Type:': 'Thermometer', 'Br...",None,None,"{'Selling Units:': 'Single item', 'Single pack...",None,{None: 'https:https://sc02.alicdn.com/kf/HTB1G...,"{'Business Type': 'Trading Company', 'Country ...",{'factory_info': {}},{},{'main_markets and products': [{'Main Markets'...,https://www.alibaba.com/product-detail/CE-ISO-...,Medical Devices
1,Multi-purpose Mini UVC Light Mobile Phone Ster...,['https://sc01.alicdn.com/kf/H11f8aee371964111...,{'Quick Details': {'Type:': 'Dry Heat Steriliz...,None,None,"{'Packaging Details': '100pcs/ctn', 'Port': 'N...",None,{'Company Information': 'https://sc01.alicdn.c...,"{'Business Type': 'Manufacturer, Trading Compa...","{'factory_info': {'Factory Size': '3,000-5,000...",{'certification': [{'Picture': 'https://s.alic...,{'main_markets and products': [{'Main Markets'...,https://www.alibaba.com/product-detail/Multi-p...,Medical Devices
2,In Stock UVC LED Sterilization Bag Sanitation ...,['https://sc01.alicdn.com/kf/Ha5387544ff5d4d0a...,{'Quick Details': {'Type:': 'Radiation Sterili...,None,None,"{'Port': 'shenzhen', 'Lead Time :': 'Quantity(...",None,{None: 'https://sc01.alicdn.com/kf/H1157afc12d...,"{'Business Type': 'Manufacturer', 'Country / R...","{'factory_info': {'Factory Size': '1,000-3,000...",{},{'main_markets and products': [{'Main Markets'...,https://www.alibaba.com/product-detail/In-Stoc...,Medical Devices
3,Highest configuration ICU ventilator S1100 Med...,['https://img.alicdn.com/imgextra/i4/600000000...,{'Quick Details': {'Type:': 'Breathing Apparat...,None,None,"{'Selling Units:': 'Single item', 'Single pack...",None,{},"{'Business Type': 'Manufacturer, Trading Compa...",{'factory_info': {'': ''}},{'certification': [{'Picture': 'https://s.alic...,{'main_markets and products': [{'Main Markets'...,https://www.alibaba.com/product-detail/Highest...,Medical Devices
4,STOCK Surgical Mask N95 Medical Portable Light...,['https://img.alicdn.com/imgextra/i3/600000000...,{'Quick Details': {'Brand Name:': 'Easternpack...,None,None,"{'Selling Units:': 'Single item', 'Single pack...",None,{'Product Description': 'https://sc01.alicdn.c...,"{'Business Type': 'Manufacturer', 'Country / R...",{'factory_info': {'': ''}},{'certification': [{'Picture': 'https://s.alic...,{'main_markets and products': [{'Main Markets'...,https://www.alibaba.com/product-detail/STOCK-S...,Medical Devices
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
462,2020 Latest Intelligent Electric Wireless Neck...,['https://img.alicdn.com/imgextra/i2/600000000...,{'Quick Details': {'Place of Origin:': 'China'...,None,None,{'Packaging Details': '2020 Latest Intelligent...,{'1.Q': ' Are you a factory or trading company...,{'Company Information': 'https://sc01.alicdn.c...,"{'Business Type': 'Manufacturer', 'Country / R...","{'factory_info': {'Factory Size': '1,000-3,000...",{'certification': [{'Picture': 'https://s.alic...,{},https://www.alibaba.com/product-detail/2020-La...,Health Care Products
463,Jinghao Disability Aids Hearing Aid Rechargeab...,['https://img.alicdn.com/imgextra/i1/600000000...,{'Quick Details': {'Place of Origin:': 'Guangd...,None,None,"{'Packaging Details': 'Jinghao brand package',...",None,{None: 'https:https://sc01.alicdn.com/kf/HTB1d...,"{'Business Type': 'Manufacturer', 'Country / R...","{'factory_info': {'Factory Size': '3,000-5,000...",{'certification': [{'Picture': 'https://s.alic...,{'main_markets and products': [{'Main Markets'...,https://www.alibaba.com/product-detail/Jinghao...,Health Care Products
464,Hot car buttocks vibration heated massage chai...,['https://img.alicdn.com/imgextra/i4/600000000...,{'Quick Details': {'Place of Origin:': 'Zhejia...,None,None,"{'Packaging Details': '1p

In [133]:
final_df1.to_csv('final_output_file.csv',index = None)

In [134]:
df.head()

,category,product_urls
0,Medical Devices,https://www.alibaba.com/product-detail/CE-ISO-...
1,Medical Devices,https://www.alibaba.com/product-detail/Multi-p...
2,Medical Devices,https://www.alibaba.com/product-detail/In-Stoc...
3,Medical Devices,https://www.alibaba.com/product-detail/Highest...
4,Medical Devices,https://www.alibaba.com/product-detail/STOCK-S...


In [124]:
final_df1.append(df,axis =1)

TypeError: append() got an unexpected keyword argument 'axis'

In [31]:
with open('all_content.json','w') as f:
    json.dump(full_json,f)